In [1]:
%load_ext autoreload
%autoreload 2

# Partition Documents

In [2]:
from aerospace_chatbot.processing import DocumentProcessor
from aerospace_chatbot.services import EmbeddingService, RerankService, LLMService, DatabaseService
from aerospace_chatbot.processing import QAModel

from langchain_core.documents import Document

# Load environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

# Initialize logger
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [3]:
embedding_service='OpenAI'
embedding_model='text-embedding-3-large'

embedding_service = EmbeddingService(
    model_service=embedding_service,
    model=embedding_model
)

doc_processor = DocumentProcessor(
    embedding_service=embedding_service
)

In [4]:
# buckets = ["ams_pdfs", "esmats_pdfs"]
buckets = ["processing-pdfs"]

docs_all = []
for bucket in buckets:
    docs_bucket = DocumentProcessor.list_bucket_pdfs(bucket)
    docs_all.append(docs_bucket)

INFO:aerospace_chatbot.processing.documents:Number of PDFs found: 414
INFO:aerospace_chatbot.processing.documents:PDFs found: ['gs://processing-pdfs/2011_alvarez_reocr.pdf', 'gs://processing-pdfs/2011_anderson_reocr.pdf', 'gs://processing-pdfs/2011_andion_reocr.pdf', 'gs://processing-pdfs/2011_arend_reocr.pdf', 'gs://processing-pdfs/2011_belly_reocr.pdf', 'gs://processing-pdfs/2011_bijuprasad_reocr.pdf', 'gs://processing-pdfs/2011_billing_reocr.pdf', 'gs://processing-pdfs/2011_blanc_reocr.pdf', 'gs://processing-pdfs/2011_bourrec_reocr.pdf', 'gs://processing-pdfs/2011_budinoff_reocr.pdf', 'gs://processing-pdfs/2011_buttery2_reocr.pdf', 'gs://processing-pdfs/2011_buttery_reocr.pdf', 'gs://processing-pdfs/2011_cavan_reocr.pdf', 'gs://processing-pdfs/2011_collado_reocr.pdf', 'gs://processing-pdfs/2011_combernoux_reocr.pdf', 'gs://processing-pdfs/2011_compostizo_reocr.pdf', 'gs://processing-pdfs/2011_courtois_reocr.pdf', 'gs://processing-pdfs/2011_demerville_reocr.pdf', 'gs://processing-pdf

In [5]:
len(docs_all)

1

In [6]:
partitioned_docs_all = []
for bucket, docs in zip(buckets, docs_all):
    # partitioned_docs = doc_processor.load_and_partition_documents(docs,partition_by_api=False, upload_bucket=bucket)
    partitioned_docs = doc_processor.load_and_partition_documents(docs,partition_by_api=False, upload_bucket=None)
    partitioned_docs_all.append(partitioned_docs)

INFO:aerospace_chatbot.processing.documents:Loading 414 documents...
INFO:aerospace_chatbot.processing.documents:Checking document 1 of 414: gs://processing-pdfs/2011_alvarez_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Downloading PDF from GCS: gs://processing-pdfs/2011_alvarez_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Bucket name: processing-pdfs
INFO:aerospace_chatbot.processing.documents:Blob name: 2011_alvarez_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Checking document 2 of 414: gs://processing-pdfs/2011_anderson_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Downloading PDF from GCS: gs://processing-pdfs/2011_anderson_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Bucket name: processing-pdfs
INFO:aerospace_chatbot.processing.documents:Blob name: 2011_anderson_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Checking document 3 of 414: gs://processing-pdfs/2011_andion_reocr.pdf
INFO:aerospace_chatbot.processing.documents:Downloa

In [ ]:
len(partitioned_docs_all)